In [1]:
"""
Scrape reviews from glassdoor using given links 
Saves individual company reviews to csv file
Method to combine all csvs into one dataframe included 
"""

'\nScrape reviews from glassdoor using given links \nSaves individual company reviews to csv file\nMethod to combine all csvs into one dataframe included \n'

In [2]:
import pandas as pd
import requests
import time
import math
import re
import pickle
import itertools
import glob

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [3]:
"""
Takes in a list of csv files using glob 
Output: dataframe combining all csvs vertically (axis=0)
Assumes all csvs have same column names 
"""
def files_to_df(glob_csv):
    final_df = pd.DataFrame()
    
    for file in glob_csv:
        df = pd.read_csv(file)
        df = df.drop(df.columns[0], axis=1)
        final_df = pd.concat([final_df, df])
        
    return final_df

In [ ]:
""" 
Load all csv files from folders 
drop each group into a dataframe 

"""
csvfiles = glob.glob('reviews/top_50_smb/' + '*.csv')
top_50_smb_reviews = files_to_df(csvfiles)
top_50_smb_reviews

csvfiles = glob.glob('reviews/top_large/' + '*.csv')
top_large_reviews =  files_to_df(csvfiles)
top_large_reviews

csvfiles = glob.glob('reviews/low_ratings_finance/' + '*.csv')
low_ratings_finance_reviews =  files_to_df(csvfiles)
low_ratings_finance_reviews

In [86]:
"""
Load links to ratings 
links_top_50_smb: links to top 50 SMB companies to work for in 2022 according to GD (based off of 2021 reviews)
links_top_large: links to top 100 large companies to work for in 2022 according to GD (based off of 2021 reviews)
links_low_ratings_finance: links to companies rated below 3 (451 total) in the NY area finance industry
# """
# with open('links_top_50_smb.pickle','rb') as f:
#     links_top_50_smb = pickle.load(f)
    
# with open('links_top_large.pickle','rb') as f:
#     links_top_large = pickle.load(f)

# with open('/glassdoor-review-scraper/links_low_ratings_finance.pickle','rb') as f:
#     links_low_ratings_finance = pickle.load(f)

# with open('links_internet.pickle','rb') as f:
#     links_internet = pickle.load(f)
    
with open('links_finance.pickle','rb') as f:
    links_finance = pickle.load(f)

In [7]:
new_links = []
for link in links_top_large:
    new_links.append(link + '?sort.sortType=RD&sort.ascending=false')
new_links

['https://www.glassdoor.com/Reviews/NVIDIA-Reviews-E7633.htm?sort.sortType=RD&sort.ascending=false',
 'https://www.glassdoor.com/Reviews/HubSpot-Reviews-E227605.htm?sort.sortType=RD&sort.ascending=false',
 'https://www.glassdoor.com/Reviews/Bain-and-Company-Reviews-E3752.htm?sort.sortType=RD&sort.ascending=false',
 'https://www.glassdoor.com/Reviews/eXp-Realty-Reviews-E907965.htm?sort.sortType=RD&sort.ascending=false',
 'https://www.glassdoor.com/Reviews/Box-Reviews-E254092.htm?sort.sortType=RD&sort.ascending=false',
 'https://www.glassdoor.com/Reviews/Boston-Consulting-Group-Reviews-E3879.htm?sort.sortType=RD&sort.ascending=false',
 'https://www.glassdoor.com/Reviews/Google-Reviews-E9079.htm?sort.sortType=RD&sort.ascending=false',
 'https://www.glassdoor.com/Reviews/Veterans-United-Home-Loans-Reviews-E242421.htm?sort.sortType=RD&sort.ascending=false',
 'https://www.glassdoor.com/Reviews/lululemon-Reviews-E42589.htm?sort.sortType=RD&sort.ascending=false',
 'https://www.glassdoor.com/Re

In [126]:
for i, link in enumerate(links_finance):
    if 'Manulife' in link:
        print(i, link)

122 https://www.glassdoor.com/Reviews/Manulife-Reviews-E9373.htm?sort.sortType=RD&sort.ascending=false


In [128]:
for company_url in reversed(links_finance[:-209]):
    print(company_url)

https://www.glassdoor.com/Reviews/Manulife-Reviews-E9373.htm?sort.sortType=RD&sort.ascending=false
https://www.glassdoor.com/Reviews/MSCI-Reviews-E14616.htm?sort.sortType=RD&sort.ascending=false
https://www.glassdoor.com/Reviews/TitleMax-Reviews-E306086.htm?sort.sortType=RD&sort.ascending=false
https://www.glassdoor.com/Reviews/Bread-Financial-Reviews-E11421.htm?sort.sortType=RD&sort.ascending=false
https://www.glassdoor.com/Reviews/Mr-Cooper-Reviews-E1748022.htm?sort.sortType=RD&sort.ascending=false
https://www.glassdoor.com/Reviews/PIMCO-Reviews-E3585.htm?sort.sortType=RD&sort.ascending=false
https://www.glassdoor.com/Reviews/Bridgewater-Associates-Reviews-E117647.htm?sort.sortType=RD&sort.ascending=false
https://www.glassdoor.com/Reviews/LendingClub-Reviews-E349061.htm?sort.sortType=RD&sort.ascending=false
https://www.glassdoor.com/Reviews/Wachovia-Reviews-E13482.htm?sort.sortType=RD&sort.ascending=false
https://www.glassdoor.com/Reviews/SEI-Investments-Reviews-E1851.htm?sort.sortTy

In [ ]:
# TODO bridgewater, Citadel 

In [ ]:
# add chrome options
chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--headless")

# initiate driver 
driver =  webdriver.Chrome(options=chrome_options) 
driver.get('https://www.glassdoor.com/index.htm') 

# NAVIGATE TO SIGN ON 
#page 1
sign_in_button= driver.find_element(By.XPATH, '//button[text()="Sign In"]')
sign_in_button.click()

# login page
username = driver.find_element(By.ID,'modalUserEmail')
username.send_keys('nicholasjang0614@gmail.com')
password = driver.find_element(By.ID,'modalUserPassword')
password.send_keys('ngtee0614')
log_in_button = driver.find_element(By.NAME, 'submit')
log_in_button.click()

# time.sleep(1)
# WAIT FOR MAIN PAGE TO LOAD AFTER LOGIN  
try:
    element = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.ID, 'Discover'))
    )
except:
    print('error loading main page.') 

reviews_companies = pd.DataFrame()

for company_url in reversed(links_finance[:-209]):
    time.sleep(5)
    
    print(f'Getting reviews for {company_url}')
    df = get_company_reviews(driver, company_url)
    # if the company doesn't have enough reviews skip 
    if len(df) > 1:
        reviews_companies = pd.concat([reviews_companies, df])
    else:
        print(f'Skipping company {company_url}')
    
reviews_companies 

Getting reviews for https://www.glassdoor.com/Reviews/Manulife-Reviews-E9373.htm?sort.sortType=RD&sort.ascending=false
Scraping page 2.
Scraping page 3.
Scraping page 4.
Scraping page 5.
Scraping page 6.
Scraping page 7.
Scraping page 8.
Scraping page 9.
Scraping page 10.
Scraping page 11.
Scraping page 12.
Scraping page 13.
Scraping page 14.
Scraping page 15.
Scraping page 16.
Scraping page 17.
Scraping page 18.
Scraping page 19.
Scraping page 20.
Scraping page 21.
Scraping page 22.
Scraping page 23.
Scraping page 24.
Scraping page 25.
Scraping page 26.
Scraping page 27.
Scraping page 28.
Scraping page 29.
Scraping page 30.
Scraping page 31.
Scraping page 32.
Scraping page 33.
Scraping page 34.
Scraping page 35.
Scraping page 36.
Scraping page 37.
Scraping page 38.
Scraping page 39.
Scraping page 40.
Scraping page 41.
Scraping page 42.
Scraping page 43.
Scraping page 44.
Scraping page 45.
Scraping page 46.
Scraping page 47.
Scraping page 48.
Scraping page 49.
Scraping page 50.
Scrapin

Scraping page 76.
Scraping page 77.
Scraping page 78.
Scraping page 79.
Scraping page 80.
Scraping page 81.
Scraping page 82.
Scraping page 83.
Scraping page 84.
Scraping page 85.
Scraping page 86.
Scraping page 87.
Scraping page 88.
Scraping page 89.
2011-12-06
Getting reviews for https://www.glassdoor.com/Reviews/Bread-Financial-Reviews-E11421.htm?sort.sortType=RD&sort.ascending=false
Scraping page 2.
Scraping page 3.
Scraping page 4.
Scraping page 5.
Scraping page 6.
Scraping page 7.
Scraping page 8.
Scraping page 9.
Scraping page 10.
Scraping page 11.
Scraping page 12.
Scraping page 13.
Scraping page 14.
Scraping page 15.
Scraping page 16.
Scraping page 17.
Scraping page 18.
Scraping page 19.
Scraping page 20.
Scraping page 21.
Scraping page 22.
Scraping page 23.
Scraping page 24.
Scraping page 25.
Scraping page 26.
Scraping page 27.
Scraping page 28.
Scraping page 29.
Scraping page 30.
Scraping page 31.
Scraping page 32.
Scraping page 33.
Scraping page 34.
Scraping page 35.
Scrapi

Scraping page 52.
Scraping page 53.
Scraping page 54.
Scraping page 55.
Scraping page 56.
Scraping page 57.
Scraping page 58.
Scraping page 59.
Scraping page 60.
Scraping page 61.
Scraping page 62.
Scraping page 63.
Scraping page 64.
Scraping page 65.
Scraping page 66.
2011-12-18
Getting reviews for https://www.glassdoor.com/Reviews/Jack-Henry-and-Associates-Reviews-E1543.htm?sort.sortType=RD&sort.ascending=false
Scraping page 2.
Scraping page 3.
Scraping page 4.
Scraping page 5.
Scraping page 6.
Scraping page 7.
Scraping page 8.
Scraping page 9.
Scraping page 10.
Scraping page 11.
Scraping page 12.
Scraping page 13.
Scraping page 14.
Scraping page 15.
Scraping page 16.
Scraping page 17.
Scraping page 18.
Scraping page 19.
Scraping page 20.
Scraping page 21.
Scraping page 22.
Scraping page 23.
Scraping page 24.
Scraping page 25.
Scraping page 26.
Scraping page 27.
Scraping page 28.
Scraping page 29.
Scraping page 30.
Scraping page 31.
Scraping page 32.
Scraping page 33.
Scraping page 3

Scraping page 82.
Scraping page 83.
Scraping page 84.
Scraping page 85.
Scraping page 86.
Scraping page 87.
Scraping page 88.
Scraping page 89.
Scraping page 90.
Scraping page 91.
Scraping page 92.
Scraping page 93.
Scraping page 94.
Scraping page 95.
Scraping page 96.
Scraping page 97.
Scraping page 98.
Scraping page 99.
Scraping page 100.
Scraping page 101.
Scraping page 102.
Scraping page 103.
Scraping page 104.
Scraping page 105.
Scraping page 106.
Scraping page 107.
Scraping page 108.
Scraping page 109.
Scraping page 110.
Scraping page 111.
Scraping page 112.
Scraping page 113.
Scraping page 114.
Scraping page 115.
Scraping page 116.
Scraping page 117.
Scraping page 118.
Scraping page 119.
Scraping page 120.
Scraping page 121.
Scraping page 122.
Scraping page 123.
Scraping page 124.
Scraping page 125.
Scraping page 126.
Scraping page 127.
Scraping page 128.
Scraping page 129.
Scraping page 130.
Scraping page 131.
Scraping page 132.
Scraping page 133.
Scraping page 134.
Scraping pa

Scraping page 165.
Scraping page 166.
Scraping page 167.
Scraping page 168.
Scraping page 169.
Scraping page 170.
Scraping page 171.
Scraping page 172.
Scraping page 173.
Scraping page 174.
Scraping page 175.
Scraping page 176.
Scraping page 177.
Scraping page 178.
Scraping page 179.
Scraping page 180.
Scraping page 181.
Scraping page 182.
Scraping page 183.
Scraping page 184.
Scraping page 185.
2011-12-28
Getting reviews for https://www.glassdoor.com/Reviews/Empower-Reviews-E25742.htm?sort.sortType=RD&sort.ascending=false
Scraping page 2.
Scraping page 3.
Scraping page 4.
Scraping page 5.
Scraping page 6.
Scraping page 7.
Scraping page 8.
Scraping page 9.
Scraping page 10.
Scraping page 11.
Scraping page 12.
Scraping page 13.
Scraping page 14.
Scraping page 15.
Scraping page 16.
Scraping page 17.
Scraping page 18.
Scraping page 19.
Scraping page 20.
Scraping page 21.
Scraping page 22.
Scraping page 23.
Scraping page 24.
Scraping page 25.
Scraping page 26.
Scraping page 27.
Scraping pa

...

Scraping page 132.
Scraping page 133.
Scraping page 134.
Scraping page 135.
Scraping page 136.
Scraping page 137.
Scraping page 138.
Scraping page 139.
Scraping page 140.
Scraping page 141.
Scraping page 142.
Scraping page 143.
Scraping page 144.
Scraping page 145.
Scraping page 146.
Scraping page 147.
Scraping page 148.
Scraping page 149.
Scraping page 150.
Scraping page 151.
Scraping page 152.
Scraping page 153.
Scraping page 154.
Scraping page 155.
Scraping page 156.
Scraping page 157.
Scraping page 158.
Scraping page 159.
Scraping page 160.
Scraping page 161.
Scraping page 162.
Scraping page 163.
Scraping page 164.
Scraping page 165.
Scraping page 166.
Scraping page 167.
Scraping page 168.
Scraping page 169.
Scraping page 170.
Scraping page 171.
Scraping page 172.
Scraping page 173.
Scraping page 174.
Scraping page 175.
Scraping page 176.
Scraping page 177.
Scraping page 178.
Scraping page 179.
Scraping page 180.
Scraping page 181.
Scraping page 182.
Scraping page 183.
Scraping pag

Scraping page 240.
Scraping page 241.
Scraping page 242.
Scraping page 243.
Scraping page 244.
Scraping page 245.
Scraping page 246.
Scraping page 247.
Scraping page 248.
Scraping page 249.
Scraping page 250.
Scraping page 251.
Scraping page 252.
Scraping page 253.
Scraping page 254.
Scraping page 255.
Scraping page 256.
Scraping page 257.
Scraping page 258.
Scraping page 259.
Scraping page 260.
Scraping page 261.
Scraping page 262.
Scraping page 263.
Scraping page 264.
Scraping page 265.
Scraping page 266.
Scraping page 267.
Scraping page 268.
Scraping page 269.
Scraping page 270.
Scraping page 271.
Scraping page 272.
Scraping page 273.
Scraping page 274.
Scraping page 275.
Scraping page 276.
Scraping page 277.
Scraping page 278.
Scraping page 279.
Scraping page 280.
Scraping page 281.
Scraping page 282.
Scraping page 283.
Scraping page 284.
Scraping page 285.
Scraping page 286.
Scraping page 287.
Scraping page 288.
Scraping page 289.
Scraping page 290.
Scraping page 291.
Scraping pag

Scraping page 32.
Scraping page 33.
Scraping page 34.
Scraping page 35.
Scraping page 36.
Scraping page 37.
Scraping page 38.
Scraping page 39.
Scraping page 40.
Scraping page 41.
Scraping page 42.
Scraping page 43.
Scraping page 44.
Scraping page 45.
Scraping page 46.
Scraping page 47.
Scraping page 48.
Scraping page 49.
Scraping page 50.
Scraping page 51.
Scraping page 52.
Scraping page 53.
Scraping page 54.
Scraping page 55.
Scraping page 56.
Scraping page 57.
Scraping page 58.
Scraping page 59.
Scraping page 60.
Scraping page 61.
Scraping page 62.
Scraping page 63.
Scraping page 64.
Scraping page 65.
Scraping page 66.
Scraping page 67.
Scraping page 68.
Scraping page 69.
Scraping page 70.
Scraping page 71.
Scraping page 72.
Scraping page 73.
Scraping page 74.
Scraping page 75.
Scraping page 76.
Scraping page 77.
Scraping page 78.
Scraping page 79.
Scraping page 80.
Scraping page 81.
Scraping page 82.
Scraping page 83.
Scraping page 84.
Scraping page 85.
Scraping page 86.
Scraping p

Scraping page 156.
Scraping page 157.
Scraping page 158.
Scraping page 159.
Scraping page 160.
Scraping page 161.
Scraping page 162.
Scraping page 163.
Scraping page 164.
Scraping page 165.
Scraping page 166.
Scraping page 167.
Scraping page 168.
Scraping page 169.
Scraping page 170.
Scraping page 171.
Scraping page 172.
Scraping page 173.
Scraping page 174.
Scraping page 175.
Scraping page 176.
Scraping page 177.
Scraping page 178.
Scraping page 179.
Scraping page 180.
Scraping page 181.
Scraping page 182.
Scraping page 183.
Scraping page 184.
Scraping page 185.
Scraping page 186.
Scraping page 187.
Scraping page 188.
Scraping page 189.
Scraping page 190.
Scraping page 191.
Scraping page 192.
Scraping page 193.
Scraping page 194.
Scraping page 195.
Scraping page 196.
Scraping page 197.
Scraping page 198.
Scraping page 199.
Scraping page 200.
Scraping page 201.
Scraping page 202.
Scraping page 203.
Scraping page 204.
Scraping page 205.
Scraping page 206.
Scraping page 207.
Scraping pag

Scraping page 244.
Scraping page 245.
Scraping page 246.
Scraping page 247.
Scraping page 248.
Scraping page 249.
Scraping page 250.
Scraping page 251.
Scraping page 252.
Scraping page 253.
Scraping page 254.
Scraping page 255.
Scraping page 256.
Scraping page 257.
Scraping page 258.
Scraping page 259.
Scraping page 260.
Scraping page 261.
Scraping page 262.
Scraping page 263.
Scraping page 264.
Scraping page 265.
Scraping page 266.
Scraping page 267.
Scraping page 268.
Scraping page 269.
Scraping page 270.
Scraping page 271.
Scraping page 272.
Scraping page 273.
Scraping page 274.
Scraping page 275.
Scraping page 276.
Scraping page 277.
Scraping page 278.
Scraping page 279.
Scraping page 280.
Scraping page 281.
Scraping page 282.
Scraping page 283.
Scraping page 284.
Scraping page 285.
Scraping page 286.
Scraping page 287.
Scraping page 288.
Scraping page 289.
Scraping page 290.
Scraping page 291.
Scraping page 292.
Scraping page 293.
Scraping page 294.
Scraping page 295.
Scraping pag

In [129]:
""" 
For a single company
Get all reviews by searching through all pages 
"""
def get_company_reviews(driver, company_url):
    try:
        driver.get(company_url)
    except Exception as e:
        print('Unable to get new URL. Sleeping for 5 mins.')
        print(e)
        time.sleep(60*5)
        driver.get(current_url)

    # WAIT FOR COMPANY PAGE TO LOAD  # time.sleep(2)   
    if wait_for_page(driver) == False:
        time.sleep(60*5)
        driver.get(current_url)

    # GET PAGE SOURCE 
    htmlSource = driver.page_source
    soup = BeautifulSoup(htmlSource,"html.parser")

    # GET NUMBER OF PAGES OF REVIEWS & company name 
    num_pages = get_num_pages(soup)
    # If < 500 reviews on site in English, skip the company 
    if num_pages < 50: 
        return pd.DataFrame()
    
    # ------
    # otherwise keep going 
    company_name = get_company_name(soup)

    # get reviews from first page 
    all_reviews = []
    first_review = get_reviews_page(soup)
    all_reviews.append(first_review)

    # if more than one page
    if num_pages > 1:
        for i in range(2,num_pages+1,1): 
            print(f'Scraping page {i}.')
            # get the new url for another page 
            current_url = get_new_url(company_url, i) 
            
            # open page and wait for loading 
            try:
                driver.get(current_url)
            except Exception as e:
                print('Unable to get new URL.')
                print(e)
                
            if wait_for_page(driver) == False:
                print('Wait for page false.')
                time.sleep(60*5)
                driver.get(current_url)
                
            # download the source of page 
            htmlSource = driver.page_source
            soup = BeautifulSoup(htmlSource,"html.parser")
            # get all reviews from page and append to all_reviews list 
            try:
                current_reviews = get_reviews_page(soup)
            except: 
                final_df = pd.DataFrame(flatten(all_reviews))
                final_df.to_csv(f'reviews/top_large/{company_name}.csv')
                return final_df 
            
            all_reviews.append(current_reviews)
            # if reviews are older than 2012 stop aggregating 
            if get_date(soup, '2012') == False:
                break
    # save to csv
    final_df = pd.DataFrame(flatten(all_reviews))
    final_df.to_csv(f'reviews/finance/{company_name}.csv')
    return final_df


"""
append sorting criteria to reviews page so reviews are filtered by date, newest first 
"""
def add_sorting_url(links):
    new_links = []
    for link in links:
        new_links.append(link + '?sort.sortType=RD&sort.ascending=false')
    return new_links


"""
Wait for page to load, wait 10 seconds before throwing error 
returns false if there is an error 
"""
def wait_for_page(driver):
    # WAIT FOR COMPANY PAGE TO LOAD 
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'EIReviews'))
        )
    except:
        print('error loading company page.')
        return False
    
    return True 

"""
Used to move to new page under company reviews
"""
def get_new_url(company_url, page_num):
    url_split = company_url.split('.')
    url_split[2] = url_split[2] + f'_P{page_num}'
    new_url = '.'.join(url_split)
    return new_url
# https://www.glassdoor.com/Reviews/KlientBoost-Reviews-E1572205.htm?sort.sortType=RD&sort.ascending=false

"""
Get the number of pages of reviews for a particular company 
"""
def get_num_pages(soup):
    try:
        footer_string = soup.find(class_='paginationFooter').text
        num_results = int(footer_string.split('of ')[1].split(' ')[0].replace(',','')) 
        num_pages = math.ceil(int(num_results/10))
    
    except:
        return 1
    
    return num_pages

"""
Get the date of the review 
If the date of the review is earlier than year, return False 
"""
def get_date(soup, year):
    reviews = soup.find_all(class_ ='empReview')
    
    for review in reviews:
        author_info = review.find(class_='authorInfo').text.split(' - ')
        date = pd.to_datetime(author_info[0]).date()
        if date < pd.to_datetime(year):
            print(date)
            return False
    
    return True

def get_company_name(soup):
    try:
        company_title = soup.find(class_='employerName').text
        company_title = company_title.replace('/','')
    except:
        company_title = soup.find('h3',{'class':'m-0 css-be8uqy el6ke056'}).text
        if company_title == None: 
            print('Could not find company title. Tried 2 methods.')
#         html = soup.prettify("utf-8")
#         with open('test.html','w',encoding='UTF-8') as file:
#             file.write(str(soup))

        return 
    
    return company_title

"""
Takes in a page's source (Soup)
Get the reviews on a single page and add to a dictionary 
A dictionary is created for each review 
returns the full list of dictionaries for that page 
"""
def get_reviews_page(soup):
    reviews = soup.find_all(class_ ='empReview')
    company_title = get_company_name(soup)
    reviews_page = []

    for review in reviews:
        review_dict = {}
        headline = review.find('h2').text
        overall_rating = float(review.find(class_='ratingNumber mr-xsm').text)

        author_info = review.find(class_='authorInfo').text.split(' - ')
        date = pd.to_datetime(author_info[0]).date()
        title = author_info[1].split('\xa0')[0]

        pros = review.find_all(class_='v2__EIReviewDetailsV2__fullWidth')[0].find_all('p')[1].text
        cons = review.find_all(class_='v2__EIReviewDetailsV2__fullWidth')[1].find_all('p')[1].text

        review_dict['company'] = company_title
        review_dict['headline'] = headline
        review_dict['date'] = date
        review_dict['overall_rating'] = overall_rating
        review_dict['author_position'] = title
        review_dict['pros'] = pros
        review_dict['cons'] = cons 

        reviews_page.append(review_dict)
        
    return reviews_page


def flatten(lst):
    return list(itertools.chain(*lst))

